<a href="https://colab.research.google.com/github/eugenie-kim012/LLMDS4/blob/main/D%2B63%2CText_Splitters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Text Splitters**

### **단순 글자수 기반 문서 분할, CharacterTextSplitter**

**[Chunk Overlap 개념 알아보기]**

In [ ]:
# #Langchain Text Splitter 모듈 다운로드
# !pip install -qU langchain langchain-community langchain-text-splitters langchain-openai langchain-experimental pypdfium2 pypdf

In [1]:
!pip install -qU langchain-text-splitters
!pip install -U langchain-community
!pip install -U langchain langchain-core

In [2]:
# 구글 드라이브 연결 (연결은 필수)

from google.colab import drive
drive.mount('/content/drive') #구글 드라이브 전체와 연결하는 함수

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install pypdfium2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.3 MB/s eta 0:00:00


In [7]:
#PyPDFium2Loader로 PDF 문서 로드하기
from langchain.document_loaders import PyPDFium2Loader
loader = PyPDFium2Loader(r"/content/drive/MyDrive/WB/2025JPO/Reading/Riding the Demographic Wave- Pensions and Retirement Income in an Aging World.pdf")
# PDF파일 불러올 객체 PyPDFLoader 선언

# PDF파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()
print(pages[6].page_content) #pages[5]페이지 내용을 객체로 긁어옴 (해당 페이지의 text 내용을 긁어올 수 있음))
pages = loader.load()

#CharacterTextSplitter 모듈 로드
from langchain_text_splitters import CharacterTextSplitter

#구분자: 줄넘김, 청크 길이: 500, 청크 오버랩: 100, length_function: 글자수
#500 글자 이하로 청크를 만드라는 말..
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
#텍스트 분할
texts = text_splitter.split_documents(pages)
print(texts[23])

vi
Contents
Abstract iii
Acknowledgments iv
Abbreviations and Glossary v
1 Aging Will Define the Evolution of Pensions and Social Insurance in the Coming Years 1
2 Aging Will Affect All Regions of the World, Many at a Rapid Pace 3
3 Contributory and Social Pensions as Instruments for Income Protection in Retirement 5
4 Pension Expenditures Are Significantly Affected by the Level of Aging 8
4.1 Mature, Well-Established Pensions Show the Largest Expenditures, but Special Regimes 
Constitute Significant Pension Costs in Less-Developed Countries 10
4.2 Expenditure on Non-Contributory Pensions Mirrors Their Coverage Target 12
5 Coverage of Pensions and Social Insurance 16
5.1 Coverage in Working Age Is Determined by Extent of Formal Employment 16
5.2 Beneficiary Coverage 18
5.3 Expanding Coverage to the Informal Sector through Voluntary Pension Schemes 21
5.4 Coverage Tends to Increase with Income for Contributory Pensions and Decrease for 
Social Pensions 24
6 Adequacy of Pension Benefits 

In [8]:
print(texts[1])

page_content='© 2025 International Bank for Reconstruction and Development / The World Bank
1818 H Street NW, Washington DC 20433 
Telephone: +1 (202) 473 1000; Internet: www.worldbank.org. 
This work is a product of the staff of The World Bank with external contributions. The findings, interpretations, and conclusions expressed in this work do not 
necessarily reflect the views of The World Bank, its Board of Executive Directors, or the governments they represent.' metadata={'producer': 'Adobe PDF Library 17.0 / PDF PT 4.7.24.2 (pdf-tools.com)', 'creator': 'Adobe InDesign 20.2 (Macintosh)', 'creationdate': '2025-04-02T02:04:20+05:30', 'title': 'World Bank Document', 'author': 'World Bank Group', 'subject': '', 'keywords': '', 'moddate': 'D:20250403092210', 'source': '/content/drive/MyDrive/WB/2025JPO/Reading/Riding the Demographic Wave- Pensions and Retirement Income in an Aging World.pdf', 'total_pages': 63, 'page': 1}


In [9]:
loader = PyPDFium2Loader(r"/content/drive/MyDrive/WB/2025JPO/Reading/Riding the Demographic Wave- Pensions and Retirement Income in an Aging World.pdf")

pages = loader.load()

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pages)
print([len(i.page_content) for i in texts])

# 각 청크의 글자수를 확인해 보기 위함

[380, 1293, 172, 935, 698, 1088, 1530, 2027, 2487, 2845, 2950, 1829, 2595, 2483, 2264, 2403, 1787, 3481, 2658, 2981, 854, 3817, 907, 2358, 2254, 2092, 3720, 3263, 1215, 3863, 3783, 3999, 1425, 2119, 2231, 3259, 1953, 1773, 2183, 1612, 2257, 1273, 2740, 2132, 2050, 671, 2792, 1364, 835, 1323, 1332, 1152, 2290, 4239, 2834, 3439, 366, 2620, 1016, 1808, 1320]


### **재귀적 문서 분할, RecursiveCharacterTextSplitter**

In [10]:
#여려개의 구분자를 재귀적으로 적용하여 분할하는 것
from langchain.document_loaders import PyPDFium2Loader

loader = PyPDFium2Loader(r"/content/drive/MyDrive/WB/2025JPO/Reading/Riding the Demographic Wave- Pensions and Retirement Income in an Aging World.pdf")

pages = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter (
    separators=["\n\n", "\n", " ", ""],
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pages)
print([len(i.page_content) for i in texts])

[380, 455, 352, 454, 83, 172, 488, 492, 140, 490, 302, 479, 495, 242, 481, 469, 441, 319, 455, 471, 490, 429, 499, 32, 474, 488, 482, 478, 407, 493, 227, 424, 408, 434, 475, 413, 490, 483, 472, 458, 447, 469, 424, 428, 459, 206, 437, 462, 470, 481, 259, 470, 415, 475, 484, 491, 487, 343, 458, 451, 422, 481, 496, 455, 181, 493, 490, 433, 481, 407, 114, 465, 457, 462, 479, 420, 472, 188, 422, 464, 480, 482, 298, 424, 440, 467, 492, 469, 421, 401, 472, 458, 482, 467, 455, 467, 456, 497, 361, 423, 467, 410, 468, 480, 443, 442, 453, 494, 456, 476, 489, 496, 478, 476, 413, 397, 493, 201, 449, 486, 148, 483, 472, 479, 408, 444, 493, 50, 429, 458, 492, 469, 489, 383, 420, 491, 407, 494, 417, 181, 486, 480, 475, 498, 471, 472, 444, 460, 471, 293, 428, 486, 440, 455, 474, 497, 469, 419, 339, 483, 475, 446, 466, 440, 423, 454, 457, 421, 485, 466, 495, 484, 166, 452, 484, 418, 470, 458, 422, 418, 457, 464, 409, 451, 479, 489, 491, 431, 468, 417, 470, 418, 483, 254, 500, 418, 463, 284, 471, 387, 47

### **문맥 파악 통한 문서 분할, Semantic Chunker**

In [15]:
!pip install pypdf
!pip install -U langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.2 MB/s eta 0:00:00


In [17]:
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.5 MB/s eta 0:00:00


In [19]:
#문맥상 연관성이 떨어지기 시작하는 부근에서 청크화
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"/content/drive/MyDrive/WB/2025JPO/Reading/Riding the Demographic Wave- Pensions and Retirement Income in an Aging World.pdf")
pages = loader.load_and_split()

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
text_splitter = SemanticChunker(OpenAIEmbeddings(openai_api_key = "your api"))

texts = text_splitter.split_documents(pages)
print("-"*100)
print("[첫번째 청크]")
print(texts[0].page_content)
print("-"*100)
print("[두번째 청크]")
print(texts[5].page_content)

----------------------------------------------------------------------------------------------------
[첫번째 청크]
SOCIAL PROTECTION
No. 2512  |  MARCH 2025
DISCUSSION PAPER
Riding the Demographic Wave: 
Pensions and Retirement 
Income in an Aging World 
Gonzalo Reyes Hartley
Miglena Abels
State of Social Protection Report 2025
The 2-Billion-Person Challenge
 Background Paper #5
 Public Disclosure Authorized
Public Disclosure Authorized
Public Disclosure Authorized
Public Disclosure Authorized
----------------------------------------------------------------------------------------------------
[두번째 청크]
The report 
concludes with recommendations for adapting pension systems to evolving economic and 
demographic realities, emphasizing the need for inclusive and sustainable solutions. JEL Codes: H55, J14, J26, I38, O15, O17
Keywords: Pension system, sustainability, coverage, aging, non-contributory pension, social 
pension, social insurance, retirement policies.


In [20]:
print([len(i.page_content) for i in texts])

[383, 259, 1032, 173, 573, 363, 491, 206, 1089, 1530, 2028, 2460, 24, 322, 2516, 2632, 312, 1821, 10, 602, 1990, 1857, 635, 1683, 670, 297, 2100, 1804, 44, 2732, 754, 904, 1814, 684, 2304, 916, 3768, 33, 11, 790, 182, 403, 1958, 2036, 257, 1505, 588, 3642, 111, 2723, 539, 1162, 108, 3782, 11, 73, 2546, 1247, 30, 3816, 115, 254, 898, 561, 315, 1804, 1250, 1016, 3156, 97, 360, 1595, 558, 1215, 721, 1491, 1403, 205, 964, 1315, 1016, 257, 2275, 471, 917, 1213, 1797, 346, 526, 142, 901, 1894, 219, 1143, 838, 1328, 1263, 140, 1068, 86, 1126, 1164, 1116, 1130, 1699, 416, 73, 1876, 1025, 2379, 848, 210, 431, 694, 153, 1479, 299, 269, 744, 533, 1328, 1097, 222]
